In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
auth.authenticate_service_account()

In [ ]:
project_id = "sensei-project-test-441408"
client = bigquery.Client(project=project_id)

In [ ]:
x = """
SELECT *
FROM `sensei-project-test-441408.Netflix.Userbase`
"""

df_userbase = client.query(x).to_dataframe()
df_userbase

In [ ]:
x = """
SELECT *
FROM `sensei-project-test-441408.Netflix.Pricing`
"""

df_pricing = client.query(x).to_dataframe()
df_pricing

In [ ]:
df_userbase.describe()

In [ ]:
df_userbase.info()

In [ ]:
df_userbase['Subscription Type'].unique()

In [ ]:
df_userbase['Join Date'] = pd.to_datetime(df_userbase['Join Date'])
df_userbase['Last Payment Date'] = pd.to_datetime(df_userbase['Last Payment Date'])

In [ ]:
dict_gender = {'Male' : 1,  'Female' : 2}
dict_device = {'Smartphone': 1, 'Laptop': 2, 'Smart TV':3, 'Tablet':4}
dict_subscription = {'Basic': 1, 'Premium': 2, 'Standard': 3}

In [ ]:
df_userbase['encode_gender'] = df_userbase['Gender'].map(dict_gender)
df_userbase['encode_device'] = df_userbase['Device'].map(dict_device)
df_userbase['encode_subscription'] = df_userbase['Subscription Type'].map(dict_subscription)
df_userbase

In [ ]:
df_userbase["device_category"] = df_userbase["Device"].astype('category').cat.codes

In [ ]:
df_userbase

In [ ]:
a = df_userbase['Age']
x = df_userbase['encode_gender']
y = df_userbase['encode_device']
z = df_userbase['encode_subscription']

In [ ]:
a.corr(z)

In [ ]:
x.corr(y)

In [ ]:
y.corr(x)

In [ ]:
combined_correlation = df_userbase[['encode_device', "Age", "encode_subscription", "encode_gender"]].corr() # Corrélation des trois colonnes sur le graphique.

# Paramètre du graphique :

plt.figure(figsize=(6,4))
sns.heatmap(combined_correlation, annot=True, cmap="Blues")

plt.title("corrélation subscription, age et device")
plt.show()

In [ ]:
df_userbase

In [ ]:
date_seuil = df_userbase['Last Payment Date'].max()
periode_seuil = date_seuil.to_period('M')
print(periode_seuil)

In [ ]:
df_userbase['is active'] = df_userbase['Last Payment Date'].dt.to_period('M') >= periode_seuil
df_userbase

In [ ]:
df_userbase["is active"].value_counts()

In [ ]:
df_userbase['signup month'] = df_userbase['Join Date'].dt.to_period('M')
df_userbase

In [ ]:
cohort_signup = df_userbase.groupby('signup month')['is active'].count().reset_index()
cohort_signup.rename(columns={"is active" : "size"}, inplace=True)
cohort_signup

In [ ]:
active_user = df_userbase.groupby('signup month')['is active'].sum().reset_index()
active_user

In [ ]:
total = pd.merge(cohort_signup,active_user, on="signup month")
total['retention_rate'] = (total['is active']/total['size'])*100
total

In [ ]:
seuil = [0, 18, 25, 35, 45, 55, 65, 200]
tags = ["-18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]

df_userbase['cat_age'] = pd.cut(df_userbase['Age'], bins=seuil, labels=tags, right=False)

df_userbase

In [ ]:
cohort_age = df_userbase.groupby("cat_age").agg(size=("User ID", "count"),active=("is active" , "sum")).reset_index()

In [ ]:
for x in range(len(cohort_age)):
  print(x)
  if cohort_age['size'].loc[x] != 0:
    cohort_age['retention'].loc[x] = (cohort_age['active'].loc[x]/cohort_age['size'].loc[x])*100
  else:
    cohort_age['retention'].loc[x] = 0


FICHER .py

In [ ]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd

auth.authenticate_service_account()
project_id = "sensei-project-test-441408"
client = bigquery.Client(project=project_id)

x = """
SELECT *
FROM `sensei-project-test-441408.Netflix.Userbase`
"""
df_userbase = client.query(x).to_dataframe()

df_userbase['Join Date'] = pd.to_datetime(df_userbase['Join Date'])
df_userbase['Last Payment Date'] = pd.to_datetime(df_userbase['Last Payment Date'])
date_seuil = df_userbase['Last Payment Date'].max()
periode_seuil = date_seuil.to_period('M')
df_userbase['is active'] = df_userbase['Last Payment Date'].dt.to_period('M') >= periode_seuil
df_userbase['signup month'] = df_userbase['Join Date'].dt.to_period('M')

cohort_signup = df_userbase.groupby('signup month')['is active'].count().reset_index()
cohort_signup.rename(columns={"is active" : "size"}, inplace=True)
active_user = df_userbase.groupby('signup month')['is active'].sum().reset_index()
total = pd.merge(cohort_signup,active_user, on="signup month")
total['retention_rate'] = (total['is active']/total['size'])*100

job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
destination_id = "sensei-project-test-441408.Import_Spotify.retention"
job = client.load_table_from_dataframe(total, destination_id, job_config=job_config)